In [1]:
%load_ext autoreload

%autoreload


import os
import pandas as pd

from setfit import SetFitModel, SetFitTrainer, Trainer
import datasets, evaluate

from src.responsabilisation_classes import responsabilisation, deresponsabilisation,neutre
from sentence_transformers.losses import CosineSimilarityLoss


import torch
from sklearn.model_selection import train_test_split


multilabel_f1_metric = evaluate.load("f1", )
multilabel_accuracy_metric = evaluate.load("accuracy")




/home/robin/env/violent_men/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'is_nltk_available' from 'transformers.utils.import_utils' (/home/robin/env/violent_men/lib/python3.9/site-packages/transformers/utils/import_utils.py)

In [4]:
data = pd.DataFrame()
data['text'] = responsabilisation + deresponsabilisation + neutre
data['label'] = ['responsabilisation']*len(responsabilisation) + ['deresponsabilisation']*len(deresponsabilisation) + ['neutre']*len(neutre)

In [7]:
train, eval = train_test_split(data, test_size=0.6, random_state=0)
train_dataset = datasets.Dataset.from_pandas(train)
eval_dataset =  datasets.Dataset.from_pandas(eval)
print("# TRAIN # \n",train.label.value_counts(normalize=False))
print("# EVAL # \n",eval.label.value_counts(normalize=False))



# TRAIN # 
 label
responsabilisation      5
deresponsabilisation    5
neutre                  2
Name: count, dtype: int64
# EVAL # 
 label
deresponsabilisation    10
responsabilisation       5
neutre                   4
Name: count, dtype: int64


In [7]:
train_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 15
})

In [8]:
class ClearCache:
    def __enter__(self):
        torch.cuda.empty_cache()

    def __exit__(self, exc_type, exc_val, exc_tb):
        torch.cuda.empty_cache()



model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
epochs = 10
num_iterations = 50


def compute_metrics(y_pred, y_test):
    return {
        "f1": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="weighted")["f1"],
        "f1_micro": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="micro")["f1"],
        #"f1_sample": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="samples")["f1"],
        "accuracy": multilabel_accuracy_metric.compute(predictions=y_pred, references=y_test)["accuracy"],
        #"top_2_accuracy": multilabel_accuracy_metric.compute(predictions=y_pred, references=y_test)["accuracy"],
    }




with ClearCache():
    model = SetFitModel.from_pretrained(model_name)
    trainer = SetFitTrainer(model = model,
                            train_dataset=train_dataset,
                            eval_dataset=eval_dataset,
                            metric=compute_metrics,
                            num_epochs=epochs,
                            num_iterations=num_iterations,
                            batch_size=32,
                            loss_class=CosineSimilarityLoss,
                        )
    trainer.train()
                        
    result = trainer.evaluate()
    print(result)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
/tmp/ipykernel_131313/2100158051.py:29: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(model = model,
Map: 100%|██████████| 12/12 [00:00<00:00, 4461.23 examples/s]
***** Running training *****
  Num unique pairs = 1200
  Batch size = 32
  Num epochs = 10
  Total optimization steps = 380
  0%|          | 0/380 [00:00<?, ?it/s]

                                            

{'embedding_loss': 0.2089, 'learning_rate': 5.263157894736843e-07, 'epoch': 0.03}


{'embedding_loss': 0.0055, 'learning_rate': 1.929824561403509e-05, 'epoch': 1.32}


{'embedding_loss': 0.0014, 'learning_rate': 1.6374269005847955e-05, 'epoch': 2.63}


{'embedding_loss': 0.0008, 'learning_rate': 1.345029239766082e-05, 'epoch': 3.95}


...

{'embedding_loss': 0.0004, 'learning_rate': 7.60233918128655e-06, 'epoch': 6.58}


{'embedding_loss': 0.0006, 'learning_rate': 4.678362573099415e-06, 'epoch': 7.89}


{'embedding_loss': 0.0003, 'learning_rate': 1.7543859649122807e-06, 'epoch': 9.21}



100%|██████████| 380/380 [01:43<00:00,  3.69it/s]
***** Running evaluation *****


{'train_runtime': 103.4643, 'train_samples_per_second': 117.528, 'train_steps_per_second': 3.673, 'epoch': 10.0}
{'f1': 0.3786967418546366, 'f1_micro': 0.3684210526315789, 'accuracy': 0.3684210526315789}


In [12]:
result

{'f1': 0.275, 'f1_micro': 0.3, 'accuracy': 0.3}